In [13]:
import numpy as np
# the layer number of each stage
stage_layer_max = [2, 4, 14, 2]
# the channel number of each layer
layer = np.array([64, 128, 256, 512])
# the descending step of channel 
step = 8 
op = ['repvgg','vgg'] # every stage
maxpool = [True,False]
ratio = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]
def search_space(stage_layer, layer):
    sum = 1
    for i in stage_layer_max:
        sum *= i
    for j in layer:
        sum *= len(ratio)
    return sum*16*16
print(search_space(stage_layer_max,layer)/1e8)

2.34881024


In [3]:
from random import choices 
choices(range(1,3))[0]

2

In [4]:
choices(ratio)[0]

0.75

In [34]:
import numpy as np
from random import choices 

# the layer number of each stage
stage_layer_max = [2, 4, 14, 2]
# the channel number of each layer
layer_num_max = [64, 128, 256, 512]
# the descending step of channel 
step = 8 
op = ['repvgg','vgg'] # every stage
maxpool = [True,False]
ratio = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]
datasets = ["cifar10","cifar100"]


# operation 
# dataset = choices(datasets)[0]
# class_num = 10 if dataset == "cifar10" else 100
import yaml
import os 
import collections
def save_dict_to_yaml(dict_value: dict, save_path: str):
    """dict保存为yaml"""
    with open(save_path, 'w') as file:
        file.write(yaml.dump(dict_value, allow_unicode=True))
 
def read_yaml_to_dict(yaml_path: str):
    with open(yaml_path) as file:
        dict_value = yaml.load(file.read(), Loader=yaml.FullLoader)
        return dict_value
savedirs = []
md_fmts = []
for n in range(12,100):
    config = {}
    # 随机生成模型
    stage_layer = [choices(range(1,stage_layer_max[0]+1))[0],choices(range(1,stage_layer_max[1]+1))[0],choices(range(1,stage_layer_max[2]+1))[0],choices(range(1,stage_layer_max[3]+1))[0]]
    # stage_layer
    stage_ratio = [choices(ratio)[0],choices(ratio)[0],choices(ratio)[0],choices(ratio)[0]]
    # stage_ratio
    with_maxpool = [choices(maxpool)[0],choices(maxpool)[0],choices(maxpool)[0],choices(maxpool)[0]]
    # with_maxpool
    op_type = [choices(op)[0],choices(op)[0],choices(op)[0],choices(op)[0],choices(op)[0]]
    dir = str(n)+'-stage-' + str(stage_layer[0]) + '_' + str(stage_layer[1]) + '_' + str(stage_layer[2]) + '_' + str(stage_layer[3]) + '-ratio-' \
        + str(stage_ratio[0]) + '_' + str(stage_ratio[1]) + '_' + str(stage_ratio[2]) + '_' + str(stage_ratio[3]) + \
            '-op-' + str(op_type[0]) + '_' + str(op_type[1]) + '_' + str(op_type[2]) + '_' + str(op_type[3]) + '_' + str(op_type[4]) + '-max-' + \
                str(with_maxpool[0]) + '_' + str(with_maxpool[1]) + '_' + str(with_maxpool[2]) + '_' + str(with_maxpool[3])
    md_fmt = "|"+ str(n)+'|' + str(stage_layer[0]) + '-' + str(stage_layer[1]) + '-' + str(stage_layer[2]) + '-' + str(stage_layer[3]) + '|' \
        + str(stage_ratio[0]) + '-' + str(stage_ratio[1]) + '-' + str(stage_ratio[2]) + '-' + str(stage_ratio[3]) + \
            '|' + str(op_type[0]) + '-' + str(op_type[1]) + '-' + str(op_type[2]) + '-' + str(op_type[3]) + '-' + str(op_type[4]) + '|' + \
                str(with_maxpool[0]) + '-' + str(with_maxpool[1]) + '-' + str(with_maxpool[2]) + '-' + str(with_maxpool[3]) + "|" * 3

    config['model'] = {'stage_layer':stage_layer,'stage_ratio': stage_ratio ,'with_maxpool':with_maxpool,'op_type':op_type,"layer_num_max":layer_num_max}
    config['train'] = {'start_epoch': 0, "epochs": 300, "warmup_epochs": 20, "warmup_lr": 1.0e-2, 
                    "lr_scheduler":{"min_lr":1.0e-5,"decay_epochs":30,"decay_rate":0.1},"batch_size":256,"loss":"crossentropy","workers":4,
                    "optimizer":{"name":"SGD","base_lr":0.01,"repvgg_lr":0.001,"momentum":0.9,"eps":1.0e-8,"betas":[0.9,0.999],"weight_decay_param":{"base_decay":5e-4,"repvgg_decay":0.05,"echo":False}},
                    "label_smoothing":0.1,"clip_grad":0.0,"ema_alpha":0.0,"ema_update_period":8,"use_l2_norm":True,
                    "no_weight_decay": "rbr_dense"}
    config['val'] = {"batch_size":128,"workers":4} 
    config['output'] = {"print_freq":100,"epoch_print_freq":1,"save_freq":20,
                        "dir": os.path.join("./log",dir), "name": str(n)}
    save_path = os.path.join("./config",dir+'.yaml')
    md_fmts.append(md_fmt)
    savedirs.append(save_path)
    save_dict_to_yaml(config,save_path)




In [37]:
# for md in md_fmts:
#     print(md)
# for savedir in savedirs:
#     print("python3 train.py --config ./config/" + savedir)

In [27]:
md_fmt = "|"+ str(n)+'|' + str(stage_layer[0]) + '-' + str(stage_layer[1]) + '-' + str(stage_layer[2]) + '-' + str(stage_layer[3]) + '|' \
        + str(stage_ratio[0]) + '-' + str(stage_ratio[1]) + '-' + str(stage_ratio[2]) + '-' + str(stage_ratio[3]) + \
            '|' + str(op_type[0]) + '-' + str(op_type[1]) + '-' + str(op_type[2]) + '-' + str(op_type[3]) + '-' + str(op_type[4]) + '|' + \
                str(with_maxpool[0]) + '-' + str(with_maxpool[1]) + '-' + str(with_maxpool[2]) + '-' + str(with_maxpool[3]) + "|" * 3
md_fmt 

'|99|2-1-5-1|0.625-1.0-0.5-0.75|vgg-vgg-vgg-vgg-repvgg|True-False-False-True|||'

In [38]:
# import os
# names = os.listdir("./config/")
# names.sort()
# for num in range(0,12):
#     print("python3 train.py --config ./config/"+names[num])

In [45]:
import os
names = os.listdir("./config/")
    # print(name)
names.sort()
for num in range(0,100):
    # print(names[num])
    name = names[num].replace(".yaml","").split("-")
    
    name.remove(name[1])
    name.remove(name[2])
    name.remove(name[3])
    name.remove(name[4])
    for i in range(len(name)):
        name[i] = name[i].replace("_","-")
    # print(name)
    # print("|"+"|".join(name)+"|"*3)
    # for n in names[0].split("-"):



In [ ]:
# 训练自动生成yaml
# loss acc train val
# 存test最好的
# loss crossentropy
# SGD
# lr 0.1，0.001（repvgg
# weight decay 0.005，0.05（repvgg
# 直接量化repvgg， repvgg block 的量化，需要考虑lr和weight decay

# 随机生成模型配置，训
# 得到小数据集
# acc predictor， 是配置 （nas + predictor
# 搜索算法 RL EL （chip memory 约束项，要提acc，降latency
# 硬件性能 
# hardware performance 
# 硬件结构固定，公式计算硬件性能
# few-shot learning
# meta-learning
# 搜索考虑memory

# 1. search space
# 2. 重点是predictor
# 3. 

# random genea
# 进一步， 搜索对不同硬件都高效的网络
# 



In [103]:
# def read_yaml_to_dict(yaml_path: str):
#     with open(yaml_path) as file:
#         dict_value = yaml.load(file.read(), Loader=yaml.FullLoader)
#         return dict_value
cfg = read_yaml_to_dict("/home/qhy/Reserach/AICAS/config/stage-2_3_12_1-ratio-0.625_0.125_0.125_0.75-op-vgg_vgg_vgg_vgg-max-False_True_True_False.yaml")


In [107]:
cfg['train']["optimizer"]

{'base_lr': 0.1,
 'betas': [0.9, 0.999],
 'eps': 1e-08,
 'momentum': 0.9,
 'name': 'SGD',
 'repvgg_lr': 0.001,
 'weight_decay_param': {'base_decay': 0.005,
  'echo': False,
  'repvgg_decay': 0.05}}

In [7]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
import numpy as np

/home/qhy/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
def make_layer(stage_num,layer_num, channel_num_in , channel_num_out, op_type, with_maxpool):
    channel_nums_in = [channel_num_in]+[channel_num_out]*(layer_num-1)
    layers = []
    if with_maxpool == True:
        layers.append(("maxpool",nn.MaxPool2d(2,2)))
        if op_type == 'vgg':
            layers.append(("stage_{}_0_vgg".format(stage_num),VGGBlock(channel_num_in,channel_num_out,kernel_size=3,stride=1)))
            layers += [("stage_{}_{}_vgg".format(stage_num,i),VGGBlock(channel_num_out,channel_num_out,3)) for i in range(1,layer_num)]
        else:
            layers.append(("stage_{}_0_repvgg".format(stage_num),RepVGGBlock(channel_num_in,channel_num_out,kernel_size=3,stride=1,padding=1)))
            layers += [("stage_{}_{}_repvgg".format(stage_num,i),RepVGGBlock(channel_num_out,channel_num_out,kernel_size=3,stride=1,padding=1)) for i in range(1,layer_num)]
            
    else :
        if op_type == 'vgg':
            layers.append(("stage_{}_0_vgg".format(stage_num),VGGBlock(channel_num_in,channel_num_out,kernel_size=3,stride=2)))
            layers += [("stage_{}_{}_vgg".format(stage_num,i),VGGBlock(channel_num_out,channel_num_out,3)) for i in range(1,layer_num)]
        else:
            layers.append(("stage_{}_0_repvgg".format(stage_num),RepVGGBlock(channel_num_in,channel_num_out,kernel_size=3,stride=2))) 
            layers += [("stage_{}_{}_repvgg".format(stage_num,i),RepVGGBlock(channel_num_out,channel_num_out,kernel_size=3,stride=1,padding=1)) for i in range(1,layer_num)]
    return nn.Sequential(OrderedDict(layers))
            
        
# make_layer(1,3,3,6,'repvgg',True)

In [43]:
cfg['model']


{'layer_num_max': [64, 128, 256, 512],
 'op_type': ['repvgg', 'vgg', 'vgg', 'repvgg'],
 'stage_layer': [1, 1, 7, 1],
 'stage_ratio': [0.625, 0.75, 0.125, 0.875],
 'with_maxpool': [True, False, True, True]}

In [112]:
import torch
class Net(nn.Module):
    def __init__(self,config,num_classes):
        super(Net, self).__init__()
        self.stage_0 = make_layer(0,config["model"]["stage_layer"][0],3,int(config["model"]["layer_num_max"][0]*config["model"]["stage_ratio"][0]),config["model"]["op_type"][0],config["model"]["with_maxpool"][0])
        self.stage_1 = make_layer(1,config["model"]["stage_layer"][1],int(config["model"]["layer_num_max"][0]*config["model"]["stage_ratio"][0]),int(config["model"]["layer_num_max"][1]*config["model"]["stage_ratio"][1]),config["model"]["op_type"][1],config["model"]["with_maxpool"][1])
        self.stage_2 = make_layer(2,config["model"]["stage_layer"][2],int(config["model"]["layer_num_max"][1]*config["model"]["stage_ratio"][1]),int(config["model"]["layer_num_max"][2]*config["model"]["stage_ratio"][2]),config["model"]["op_type"][2],config["model"]["with_maxpool"][2])
        self.stage_3 = make_layer(3,config["model"]["stage_layer"][3],int(config["model"]["layer_num_max"][2]*config["model"]["stage_ratio"][2]),int(config["model"]["layer_num_max"][3]*config["model"]["stage_ratio"][3]),config["model"]["op_type"][3],config["model"]["with_maxpool"][3])
        self.gap = nn.AdaptiveAvgPool2d(output_size=1)
        self.linear = nn.Linear(int(config["model"]["layer_num_max"][3]*config["model"]["stage_ratio"][3]), num_classes)
    def forward(self, input):
        out = self.stage_0(input)
        for stage in (self.stage_1, self.stage_2,  self.stage_3):
            for block in stage:
                out = block(out)
        out = self.gap(out)
        out = self.linear(out.view(out.size(0), -1))
        return out
n = Net(cfg,10)
inp = torch.randn(1,3,32,32)
o = n(inp)
print(o.shape)

torch.Size([1, 10])


In [100]:
repvgg_stage = torch.nn.ModuleList()
net = n
for stage in (net.stage_0,net.stage_1,net.stage_2,net.stage_3):
    for name, module in stage._modules.items():
        if "repvgg" in name:
            repvgg_stage.append(stage)
repvgg_stage_params = list(map(id,repvgg_stage.parameters()))
base_stage_param = filter(lambda p:id(p) not in repvgg_stage_params,net.parameters())
optimizer = torch.optim.SGD([{'params':base_stage_param,'lr':0.1,"weight_decay":0.005},
                             {'params':repvgg_stage.parameters(),'lr':0.001,"weight_decay":0.05}])


In [113]:
# optimizer

In [90]:
for stage in (n.stage_0,n.stage_1,n.stage_2,n.stage_3):
    for name, module in stage._modules.items():
        if "repvgg" in name:
            module.switch_to_deploy()

In [92]:
# n

In [93]:
# type(n.stage_0.stage_0_0_repvgg)
# for i in n.stage_0.modules():
#     print(i.name)
#     break
# for name, module in n.stage_0._modules.items():
#     print(module)
# for name, module in n.stage_0._modules.items():
#     # print("名称:{}".format(name))
#     if "repvgg" in name:
#         module.switch_to_deploy()
# for name, module in n.stage_0._modules.items():
#     # print(name)
#     print(module)
    

In [50]:
n.stage_0.stage_0_0.switch_to_deploy()

In [ ]:
    # def switch_to_deploy(self):
    #     if hasattr(self, 'rbr_reparam'):
    #         return
    #     kernel, bias = self.get_equivalent_kernel_bias()
    #     self.rbr_reparam = nn.Conv2d(in_channels=self.rbr_dense.conv.in_channels, out_channels=self.rbr_dense.conv.out_channels, kernel_size=self.rbr_dense.conv.kernel_size, stride=self.rbr_dense.conv.stride, padding=self.rbr_dense.conv.padding, dilation=self.rbr_dense.conv.dilation, groups=self.rbr_dense.conv.groups, bias=True)
    #     self.rbr_reparam.weight.data = kernel
    #     self.rbr_reparam.bias.data = bias
    #     self.__delattr__('rbr_dense')
    #     self.__delattr__('rbr_1x1')
    #     if hasattr(self, 'rbr_identity'):
    #         self.__delattr__('rbr_identity')
    #     if hasattr(self, 'id_tensor'):
    #         self.__delattr__('id_tensor')
    #     self.deploy = True

In [32]:
def conv_bn(in_channels, out_channels, kernel_size, stride, padding, groups=1):
    result = nn.Sequential()
    result.add_module('conv', nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups, bias=False))
    result.add_module('bn', nn.BatchNorm2d(num_features=out_channels))
    return result

In [12]:
import torch.nn as nn
from collections import OrderedDict 
def VGGBlock(in_channels, out_channels, kernel_size, stride = 1, padding = 1, dilation = 1, groups = 1, padding_mode = 'zeros'):
    conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1 ,dilation = 1, groups = 1, padding_mode = 'zeros')
    layers = nn.Sequential(OrderedDict([
      ("conv",conv2d),
      ("bn",nn.BatchNorm2d(out_channels)),
      ("relu",nn.ReLU(inplace=True))]))
    return layers
VGGBlock(3,6,3)

# conv_module = nn.Sequential(OrderedDict([
#           ('conv1', nn.Conv2d(1,20,5)),
#           ('relu1', nn.ReLU()),
#           ('conv2', nn.Conv2d(20,64,5)),
#           ('relu2', nn.ReLU())
#         ]))
# conv_module 

Sequential(
  (conv): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
)

In [21]:
[i for i in range(8)]

[0, 1, 2, 3, 4, 5, 6, 7]

In [114]:

# nn.Sequential(OrderedDict([
#     ("stage_0_{}".format(i),VGGBlock(3,6,3)) for i in range(3)
    
#     ]))

In [20]:
a = nn.Sequential(OrderedDict([
    ("stage_0_1",VGGBlock(3,6,3)),
    ("stage_0_2",VGGBlock(6,6,3)),
    ]))
b = torch.randn(1,3,32,32)
c = a(b)
c.shape

torch.Size([1, 6, 32, 32])

In [25]:
# stage = nn.Sequential()
# stage.append(OrderedDict([("stage_0",VGGBlock(3,6,3))]))
# # stage.append(VGGBlock(6,6,3))
# stage

In [31]:
class RepVGGBlock(nn.Module):
    """
        Naive RepVGG without SE module
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, padding_mode='zeros', deploy=False):
        super(RepVGGBlock, self).__init__()
        self.deploy = deploy
        self.groups = groups
        self.in_channels = in_channels

        self.non_linearity = nn.ReLU()

        assert kernel_size == 3
        assert padding == 1

        padding_11 = padding - kernel_size // 2

        if deploy:
            self.rbr_reparam = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=True, padding_mode=padding_mode)
        else:
            self.rbr_identity = nn.BatchNorm2d(num_features=out_channels) if out_channels == in_channels and stride == 1 else None
            self.rbr_dense = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups)
            self.rbr_1x1 = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, padding=padding_11, groups=groups)
    
    def forward(self, inputs):
        if hasattr(self, 'rbr_reparam'):
            return self.non_linearity(self.rbr_reparam(inputs))
        
        if self.rbr_identity is None:
            id_out = 0
        else:
            id_out = self.rbr_identity(inputs)
        
        return self.non_linearity(self.rbr_dense(inputs) + self.rbr_1x1(inputs) + id_out)
    
    def get_custom_L2(self):
        K3 = self.rbr_dense.conv.weight
        K1 = self.rbr_1x1.conv.weight
        t3 = (self.rbr_dense.bn.weight / ((self.rbr_dense.bn.running_var + self.rbr_dense.bn.eps).sqrt())).reshape(-1, 1, 1, 1).detach()
        t1 = (self.rbr_1x1.bn.weight / ((self.rbr_1x1.bn.running_var + self.rbr_1x1.bn.eps).sqrt())).reshape(-1, 1, 1, 1).detach()

        l2_loss_circle = (K3 ** 2).sum() - (K3[:, :, 1:2, 1:2] ** 2).sum()      # The L2 loss of the "circle" of weights in 3x3 kernel. Use regular L2 on them.
        eq_kernel = K3[:, :, 1:2, 1:2] * t3 + K1 * t1                           # The equivalent resultant central point of 3x3 kernel.
        l2_loss_eq_kernel = (eq_kernel ** 2 / (t3 ** 2 + t1 ** 2)).sum()        # Normalize for an L2 coefficient comparable to regular L2.
        return l2_loss_eq_kernel + l2_loss_circle

    def get_equivalent_kernel_bias(self):
        kernel3x3, bias3x3 = self._fuse_bn_tensor(self.rbr_dense)
        kernel1x1, bias1x1 = self._fuse_bn_tensor(self.rbr_1x1)
        kernelid, biasid = self._fuse_bn_tensor(self.rbr_identity)
        return kernel3x3 + self._pad_1x1_to_3x3_tensor(kernel1x1) + kernelid, bias3x3 + bias1x1 + biasid

    def _pad_1x1_to_3x3_tensor(self, kernel1x1):
        if kernel1x1 is None:
            return 0
        else:
            return torch.nn.functional.pad(kernel1x1, [1, 1, 1, 1])
    
    def _fuse_bn_tensor(self, branch):
        if branch is None:
            return 0, 0
        if isinstance(branch, nn.Sequential):
            kernel = branch.conv.weight
            running_mean = branch.bn.running_mean
            running_var = branch.bn.running_var
            gamma = branch.bn.weight
            beta = branch.bn.bias
            eps = branch.bn.eps
        else:
            assert isinstance(branch, nn.BatchNorm2d)
            if not hasattr(self, 'id_tensor'):
                input_dim = self.in_channels // self.groups
                kernel_value = np.zeros((self.in_channels, input_dim, 3, 3), dtype=np.float32)
                for i in range(self.in_channels):
                    kernel_value[i, i % input_dim, 1, 1] = 1
                self.id_tensor = torch.from_numpy(kernel_value).to(branch.weight.device)
            kernel = self.id_tensor
            running_mean = branch.running_mean
            running_var = branch.running_var
            gamma = branch.weight
            beta = branch.bias
            eps = branch.eps
        std = (running_var + eps).sqrt()
        t = (gamma / std).reshape(-1, 1, 1, 1)
        return kernel * t, beta - running_mean * gamma / std
    
    def switch_to_deploy(self):
        if hasattr(self, 'rbr_reparam'):
            return
        kernel, bias = self.get_equivalent_kernel_bias()
        self.rbr_reparam = nn.Conv2d(in_channels=self.rbr_dense.conv.in_channels, out_channels=self.rbr_dense.conv.out_channels, kernel_size=self.rbr_dense.conv.kernel_size, stride=self.rbr_dense.conv.stride, padding=self.rbr_dense.conv.padding, dilation=self.rbr_dense.conv.dilation, groups=self.rbr_dense.conv.groups, bias=True)
        self.rbr_reparam.weight.data = kernel
        self.rbr_reparam.bias.data = bias
        self.__delattr__('rbr_dense')
        self.__delattr__('rbr_1x1')
        if hasattr(self, 'rbr_identity'):
            self.__delattr__('rbr_identity')
        if hasattr(self, 'id_tensor'):
            self.__delattr__('id_tensor')
        self.deploy = True

In [32]:
import torch
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.stage_1 = RepVGGBlock(3,3,3,padding=1)
        self.stage_2 = VGGBlock(3,6,3)
    def forward(self, input):
        out = self.stage_1(input)
        out = self.stage_2(out)
        return out
n = Net()
inp = torch.randn(1,3,32,32)
o = n(inp)
print(o.shape)


torch.Size([1, 6, 32, 32])


In [2]:
# RepVGGBlock(3,64,3,padding=1)

In [ ]:
# padding == 1 是为了保证尺寸不变


In [22]:
cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}
def make_layers(cfg, batch_norm=False):
    '''
    根据配置表，返回模型层列表
    '''
    layers = [] # 层列表初始化

    in_channels = 3 # 输入3通道图像

    # 遍历配置列表
    for v in cfg:
        if v == 'M': # 添加池化层
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else: # 添加卷积层

            # 3×3 卷积
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)

            # 卷积-->批归一化（可选）--> ReLU激活
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]

            # 通道数方面，下一层输入即为本层输出
            in_channels = v

    # 以sequencial类型返回模型层列表
    return nn.Sequential(*layers)

# make_layers(cfgs['A'], batch_norm=True)

In [21]:
class VGG(nn.Module):
    '''
    VGG通用网络模型
    输入features为网络的特征提取部分网络层列表
    分类数为 1000
    '''
    def __init__(self, features, dataset = 'cifar10', init_weights=True):
        super(VGG, self).__init__()
        if dataset == 'cifar10':
            num_classes = 10
        elif dataset == 'cifar100':
            num_classes = 100
            
        # 特征提取部分
        self.features = features

        # 自适应平均池化，特征图池化到 7×7 大小
        # 为了让下一步的flatten得到一个固定长度的向量
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        # 分类部分
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),   # 512*7*7 --> 4096
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),          # 4096 --> 4096
            nn.ReLU(True),
            # nn.Dropout(),
            nn.Linear(4096, num_classes),   # 4096 --> 1000
        )

        # 权重初始化
        if init_weights:
            self._initialize_weights()

    def forward(self, x):

        # 特征提取
        x = self.features(x)
        # 自适应平均池化
        x = self.avgpool(x)
        # 特征图展平成向量
        x = torch.flatten(x, 1)
        # 分类器分类输出
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        '''
        权重初始化
        '''
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 卷积层使用 kaimming 初始化
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
                # 偏置初始化为0
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            # 批归一化层权重初始化为1 
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            # 全连接层权重初始化
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [3]:
# models.vgg19_bn(pretrained=True)
'''VGG11/13/16/19 in Pytorch.'''
import torch
import torch.nn as nn


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, 10)
        self.gap = nn.AvgPool2d(kernel_size=1, stride=1)

    def forward(self, x):
        out = self.features(x)
        print(out.shape)
        out = self.gap(out)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        # layers += []
        return nn.Sequential(*layers)


def test():
    net = VGG('VGG16')
    x = torch.randn(2,3,32,32)
    y = net(x)
    print(y.size())
test()

torch.Size([2, 512, 1, 1])
torch.Size([2, 10])


In [1]:
# net = VGG('VGG16')
# net
# # {64->64:1,max,64->128:1,128->128:1,max,128->256:1,256->256:2,max,256->512:1,512:5,max,gap,}

In [3]:
# from models import model
# m = model.Net(config=config,num_classes=10)
# # x = torch.randn(2,3,32,32)
# # m(x)
# m

In [8]:
# from models import model
# m = model.Net(config=config,num_classes=10)
# x = torch.randn(2,3,32,32)
# m(x)

In [10]:
dir = str(n)+'-stage-' + str(stage_layer[0]) + '_' + str(stage_layer[1]) + '_' + str(stage_layer[2]) + '_' + str(stage_layer[3]) + '_' + str(stage_layer[4]) + '-ratio-' \
    + str(stage_ratio[0]) + '_' + str(stage_ratio[1]) + '_' + str(stage_ratio[2]) + '_' + str(stage_ratio[3]) + '_' + str(stage_ratio[4]) +  \
        '-op-' + str(op_type[0]) + '_' + str(op_type[1]) + '_' + str(op_type[2]) + '_' + str(op_type[3]) + '_' + str(op_type[4]) + '-max-' + \
            str(with_maxpool[0]) + '_' + str(with_maxpool[1]) + '_' + str(with_maxpool[2]) + '_' + str(with_maxpool[3]) + '_' + str(with_maxpool[4]) +'-'+ dataset
dir

'459-stage-1_3_1_5_1-ratio-0.5_0.5_0.75_0.875_0.625-op-repvgg_vgg_vgg_repvgg_vgg-max-False_True_True_True_True-cifar10'

In [2]:
import numpy as np
from random import choices 

# the layer number of each stage
# stage_layer_max = [1, 2, 4, 14, 1]
stage_layer_max = [2, 4, 4, 4, 4]
# the channel number of each layer
layer_num_max = [64, 64, 128, 256, 512]
# the descending step of channel 
step = 8 
op = ['repvgg','vgg'] # every stage
maxpool = [True,False]
ratio = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]
datasets = ["cifar10"]


# operation 
# class_num = 10 if dataset == "cifar10" else 100
import yaml
import os 
import collections
def save_dict_to_yaml(dict_value: dict, save_path: str):
    """dict保存为yaml"""
    with open(save_path, 'w') as file:
        file.write(yaml.dump(dict_value, allow_unicode=True))
 
def read_yaml_to_dict(yaml_path: str):
    with open(yaml_path) as file:
        dict_value = yaml.load(file.read(), Loader=yaml.FullLoader)
        return dict_value
savedirs = []
md_fmts = []
for n in range(551,640):
    config = {}
    # 随机生成模型
    dataset = choices(datasets)[0]
    
    stage_layer = [choices(range(1,stage_layer_max[0]+1))[0],choices(range(1,stage_layer_max[1]+1))[0],choices(range(1,stage_layer_max[2]+1))[0],choices(range(1,stage_layer_max[3]+1))[0],choices(range(1,stage_layer_max[4]+1))[0]]
    # stage_layer
    ratio_1 = choices(ratio)[0]
    stage_ratio = [ratio_1,ratio_1,choices(ratio)[0],choices(ratio)[0],choices(ratio)[0]]
    # stage_ratio = [1.0] * 5
    # stage_ratio
    with_maxpool = [False,choices(maxpool)[0],choices(maxpool)[0],choices(maxpool)[0],choices(maxpool)[0]]
    # with_maxpool
    op_type = [choices(op)[0],choices(op)[0],choices(op)[0],choices(op)[0],choices(op)[0]]
    # op_type = ['repvgg'] * 5
    dir = str(n)+'-stage-' + str(stage_layer[0]) + '_' + str(stage_layer[1]) + '_' + str(stage_layer[2]) + '_' + str(stage_layer[3]) + '_' + str(stage_layer[4]) + '-ratio-' \
    + str(stage_ratio[0]) + '_' + str(stage_ratio[1]) + '_' + str(stage_ratio[2]) + '_' + str(stage_ratio[3]) + '_' + str(stage_ratio[4]) +  \
        '-op-' + str(op_type[0]) + '_' + str(op_type[1]) + '_' + str(op_type[2]) + '_' + str(op_type[3]) + '_' + str(op_type[4]) + '-max-' + \
            str(with_maxpool[0]) + '_' + str(with_maxpool[1]) + '_' + str(with_maxpool[2]) + '_' + str(with_maxpool[3]) + '_' + str(with_maxpool[4]) +'-'+ dataset
    md_fmt = "|"+ str(n)+'|' + str(stage_layer[0]) + '-' + str(stage_layer[1]) + '-' + str(stage_layer[2]) + '-' + str(stage_layer[3]) + '-' + str(stage_layer[4]) + '|'\
        + str(stage_ratio[0]) + '-' + str(stage_ratio[1]) + '-' + str(stage_ratio[2]) + '-' + str(stage_ratio[3]) + '-' + str(stage_ratio[4]) + \
            '|' + str(op_type[0]) + '-' + str(op_type[1]) + '-' + str(op_type[2]) + '-' + str(op_type[3]) + '-' + str(op_type[4]) + '|' + \
                str(with_maxpool[0]) + '-' + str(with_maxpool[1]) + '-' + str(with_maxpool[2]) + '-' + str(with_maxpool[3]) + '-' + str(with_maxpool[4]) +'|'+ dataset+'|' "|" * 3

    config['model'] = {'stage_layer':stage_layer,'stage_ratio': stage_ratio ,'with_maxpool':with_maxpool,'op_type':op_type,"layer_num_max":layer_num_max}
    config['train'] = {'start_epoch': 0, "epochs": 100, "warmup_epochs": 0, "warmup_lr": 1.0e-2, 
                    "lr_scheduler":{"min_lr":1.0e-5,"decay_epochs":30,"decay_rate":0.1},"batch_size":256,"loss":"crossentropy","workers":4,
                    "optimizer":{"name":"SGD","base_lr":0.1,"repvgg_lr":0.1,"momentum":0.9,"eps":1.0e-8,"betas":[0.9,0.999],"weight_decay_param":{"base_decay":5e-4,"repvgg_decay":1e-4,"echo":False}},
                    "label_smoothing":0.1,"clip_grad":0.0,"ema_alpha":0.0,"ema_update_period":8,"use_l2_norm":True,
                    "no_weight_decay": ["rbr_dense","rbr_1x1"]}
    config['val'] = {"batch_size":128,"workers":4} 
    config['output'] = {"print_freq":100,"epoch_print_freq":1,"save_freq":20,
                        "dir": os.path.join("./log",dir), "name": str(n)}
    config['dataset'] = dataset
    save_path = os.path.join("../config",dir+'.yaml')
    md_fmts.append(md_fmt)
    savedirs.append(save_path.replace("..","."))
    save_dict_to_yaml(config,save_path)

print("="*100)
for md in md_fmts:
    print(md)
print("="*100)
for savedir in savedirs:
    print("python3 train.py --config " + savedir)

|530|1-4-3-1-1|0.75-0.75-1.0-0.625-0.625|repvgg-vgg-vgg-repvgg-vgg|False-True-False-True-True|cifar10||||||
|531|1-1-2-4-1|0.5-0.5-0.75-0.375-0.75|repvgg-vgg-repvgg-repvgg-repvgg|False-True-False-True-False|cifar10||||||
|532|1-1-3-2-1|1.0-1.0-0.75-0.625-0.125|repvgg-repvgg-vgg-repvgg-vgg|False-True-True-True-True|cifar10||||||
|533|1-2-6-4-1|0.5-0.5-0.125-0.625-0.625|vgg-repvgg-vgg-vgg-repvgg|False-False-True-True-True|cifar10||||||
|534|1-1-2-7-1|0.5-0.5-0.75-0.875-0.625|repvgg-repvgg-vgg-repvgg-repvgg|False-True-False-False-True|cifar10||||||
|535|1-2-1-8-1|0.125-0.125-0.875-1.0-1.0|vgg-repvgg-vgg-repvgg-vgg|False-False-False-True-True|cifar10||||||
|536|1-2-5-4-1|0.375-0.375-0.125-0.5-0.5|vgg-vgg-vgg-repvgg-repvgg|False-False-False-False-True|cifar10||||||
|537|1-4-5-6-1|0.625-0.625-1.0-1.0-0.25|repvgg-vgg-repvgg-repvgg-repvgg|False-True-False-False-True|cifar10||||||
|538|1-1-5-3-1|0.5-0.5-0.625-0.75-1.0|repvgg-vgg-vgg-vgg-vgg|False-False-False-False-True|cifar10||||||
|539|1-2-4-

In [11]:
from models import model
m = model.Net(config=config,num_classes=10)
# x = torch.randn(2,3,32,32)
# m(x)
from easydict import EasyDict
config = EasyDict(yaml.full_load(open("./config/13-stage-1_1_2_12_1-ratio-0.5_0.5_0.75_0.5_0.75-op-vgg_vgg_vgg_repvgg_vgg-max-False_True_True_True_False.yaml")))
config.train.optimizer.weight_decay_param.echo = True
import torch.optim as optim

def build_optimizer(config, model):
    """
    Build optimizer, set weight decay of normalization to 0 by default.
    """
    skip = {}
    skip_keywords = {}
    if hasattr(model, 'no_weight_decay'):
        skip = model.no_weight_decay()
    if hasattr(model, 'no_weight_decay_keywords'):
        if config.train.use_l2_norm:
            skip_keywords = model.no_weight_decay_keywords(
                config.train.no_weight_decay)
        else:
            skip_keywords = model.no_weight_decay_keywords()
    parameters = set_weight_decay(
        model,
        skip,
        skip_keywords,
        echo=config.train.optimizer.weight_decay_param.echo)
    repvgg_stage = torch.nn.ModuleList()
    for stage in (model.stage_0, model.stage_1, model.stage_2, model.stage_3):
        for name, module in stage._modules.items():
            if "repvgg" in name:
                repvgg_stage.append(stage)
    repvgg_stage_params = list(map(id, repvgg_stage.parameters()))
    base_stage_param = filter(lambda p: id(p) not in repvgg_stage_params,
                              model.parameters())

    opt_lower = config.train.optimizer.name.lower()
    optimizer = None
    if opt_lower == 'sgd':
        optimizer = optim.SGD(
            [{
                'params':
                base_stage_param,
                'lr':
                config.train.optimizer.base_lr,
                "weight_decay":
                config.train.optimizer.weight_decay_param.base_decay
            }, {
                'params':
                repvgg_stage.parameters(),
                'lr':
                config.train.optimizer.repvgg_lr,
                "weight_decay":
                config.train.optimizer.weight_decay_param.repvgg_decay
            }],
            momentum=config.train.optimizer.momentum,
            nesterov=True,
        )
        # optimizer = optim.SGD(parameters, momentum=config.train.optimizer.momentum, nesterov=True,
        #                       lr=config.train.optimizer.base_lr, weight_decay=config.train.optimizer.weight_decay_param.decay)
        if config.train.optimizer.weight_decay_param.echo:
            print(
                '================================== SGD nest, momentum = {}, wd = {}'
                .format(config.train.optimizer.momentum,
                        config.train.optimizer.weight_decay_param.base_decay))
    return optimizer
def set_weight_decay(model, skip_list=(), skip_keywords=(), echo=False):
    has_decay = []
    no_decay = []

    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue  # frozen weights
        if 'identity.weight' in name:
            has_decay.append(param)
            if echo:
                print(f"{name} USE weight decay")
        elif len(param.shape) == 1 or name.endswith(".bias") or (
                name in skip_list) or check_keywords_in_name(
                    name, skip_keywords):
            no_decay.append(param)
            if echo:
                print(f"{name} has no weight decay")
        else:
            has_decay.append(param)
            if echo:
                print(f"{name} USE weight decay")

    return [{'params': has_decay}, {'params': no_decay, 'weight_decay': 0.}]
def check_keywords_in_name(name, keywords=()):
    isin = False
    for keyword in keywords:
        if keyword in name:
            isin = True
    return isin
# build_optimizer(config, m)